In [ ]:
# project imports:
# !pip install geopandas # uncomment if need to install geopandas
#import geopandas
!pip install geojson # uncomment if need to install geojson
import geojson
import json
import requests
import numpy as np

In [ ]:
# count the parking spots members:
empty_pk_spots = {}
occupied_pk_spots = {}

In [ ]:
# open the file using geopandas
file_path = "parking_data/parking.geojson"
# file = open(file_path)
# data_file = geopandas.read_file(file)


# open the file using geojson (less updated)
with open(file_path) as f:
    data_file = geojson.load(f)
    # features = gj['features'][0]

# another way it should work (like any other json file)
# data = json.loads(file_path)
# data['features'][0]['geometry']


In [ ]:
print(data_file)

       fid              label  \
0        1  PK-space-occupied   
1        2  PK-space-occupied   
2        3  PK-space-occupied   
3        4  PK-space-occupied   
4        5  PK-space-occupied   
...    ...                ...   
3300  3302  PK-space-occupied   
3301  3303  PK-space-occupied   
3302  3304  PK-space-occupied   
3303  3305  PK-space-occupied   
3304  3306  PK-space-occupied   

                                                polygon  \
0     MULTIPOLYGON (((391008.0877934272 5821088.3568...   
1     MULTIPOLYGON (((391014.9892018779 5821079.7652...   
2     MULTIPOLYGON (((391025.0830985915 5821069.2018...   
3     MULTIPOLYGON (((391030.0596244131 5821058.8732...   
4     MULTIPOLYGON (((391035.8342723005 5821051.4084...   
...                                                 ...   
3300  MULTIPOLYGON (((391953.6 5824864.95, 391954.45...   
3301  MULTIPOLYGON (((391976.9 5824862.65, 391977.8 ...   
3302  MULTIPOLYGON (((391953.6 5824873.35, 391955.1 ...   
3303  MULTIPO

In [ ]:
features_array = data_file['features']

for feature in features_array:
  feature_label = feature['properties']['label'] # get spot label "occupied/empty"
  feature_coordinates = feature['geometry']['coordinates'] # get spot coordinates

  # get the OSM node-id that match the given coordinates
  spot_long = feature_coordinates[0]
  spot_lat = feature_coordinates[1]

  # get the way id - request json format of the reverse method of OSM librery nominatim
  response = requests.get("https://nominatim.openstreetmap.org/reverse?format=jsonv2&lat=" + f'{spot_lat:.4f}' + "&lon=" + f'{spot_long:.4f}')
  way_id_response = json.loads(response.text)
  way_id = way_id_response ['osm_id'] 

  if (feature_label == "PK-space-occupied"): # if the current parking spot is occupied - add 1 to the occupied count in the current way
    occupied_count = occupied_pk_spots.setdefault(way_id, 0)
    occupied_pk_spots[way_id] = occupied_count + 1

  elif (feature_label == "PK-space-empty"): # if the current parking spot is empty - add 1 to the empty count in the current way
    empty_count = empty_pk_spots.setdefault(way_id, 0)
    empty_pk_spots[way_id] = empty_count + 1 


In [ ]:
def get_prob_for_node(node_id, index):
    f = open("probabilities.osm", "r")
    flag=-1
    for line in f.readlines():
        if line.startswith('\t\t<nd ref'):
            start_index = (line.index('=')) +2 #the starting of the node_id number
            end_index= (line.index('/')) -1
            if (line[start_index:end_index]) == node_id:
              flag=0  # it means that the next tag we will see we will go inside
        elif flag == 0 and line.startswith('\t\t<tag'):
          start = line.index('v')
          end = len(line) - 4
          temp_list = line[start+3:end].split(',')
          prob_list = [eval(i) for i in temp_list]
          return prob_list[index]
    return 0 #if didn't find- return 0

In [ ]:
  def prepare_data_for_model():
    data_list=[]
    relevant_index= most_common_highest_probability()
    for node_id in empty_pk_spots.keys():
      num_of_empty= empty_pk_spots[node_id]
      if node_id in occupied_pk_spots:
        num_of_occupied= occupied_pk_spots[node_id]
      else:
        num_of_occupied=0
      prob_for_empty_parking= get_prob_for_node(node_id, relevant_index)
      data_list.append([num_of_empty, num_of_occupied, prob_for_empty_parking])

    for node_id in occupied_pk_spots.keys():
      if node_id not in empty_pk_spots: # so we didn't go through it yet and the probability is 0
        prob_for_empty_parking= get_prob_for_node(node_id, relevant_index)
        data_list.append([0, occupied_pk_spots[node_id], prob_for_empty_parking])

In [ ]:
print(empty_pk_spots)

{5758790: 1, 390623189: 1, 6975869450: 4, 2441844084: 1, 1552571091: 1, 3942724528: 2, 3942724529: 1, 3034708196: 1, 3034708151: 1, 3034708149: 1, 2742938781: 2, 2730990389: 1, 2736635168: 1, 2730990330: 1, 268315957: 1, 2736635144: 1, 268098656: 1, 268315968: 2, 2736635147: 1, 147432145: 2, 173935157: 2, 2753639335: 2, 1552053966: 1, 1552053892: 1, 1552076309: 1, 1552075377: 1, 1552073580: 1, 1552075328: 1, 105786498: 1, 105785524: 2, 201198297: 2, 3287262594: 1, 3287262593: 2, 105785522: 1, 116676564: 3, 2307922683: 1, 667808345: 1, 2252768267: 1, 2252768273: 2, 301595370: 4, 8096771: 4, 3242507965: 3, 119687772: 1, 2742938792: 1, 3376817: 1, 1844993542: 1, 1845880605: 2, 3376818: 1, 4964064354: 1, 2844218746: 1, 1552075220: 1, 1552075164: 1, 1552075285: 1, 105786493: 1, 105786488: 1, 217745069: 1, 1552569102: 1, 2775407731: 1, 1552565108: 1, 2307922684: 1, 180687339: 2, 169814741: 1, 5640350672: 1, 2252782639: 1, 7167394534: 2, 7575380500: 1, 3242507969: 1, 24544413: 1, 3728865725: 

In [ ]:
print(occupied_pk_spots)

{1912625138: 1, 2904138761: 1, 1552052958: 1, 1552051077: 2, 5592220398: 2, 5758790: 1, 1552051660: 2, 1552051256: 2, 1552053428: 1, 1552053576: 2, 390623189: 3, 6975869450: 3, 53332484: 1, 53332487: 1, 147659855: 1, 1552076309: 4, 2844218749: 2, 3938839716: 1, 3938839715: 1, 1552571091: 2, 116676564: 9, 180662834: 3, 3034708161: 1, 180662836: 1, 3034708066: 2, 4155950167: 1, 3034708064: 2, 3844356465: 1, 3844356478: 1, 3034708061: 1, 213113204: 1, 3034708196: 3, 3034708200: 1, 391005378: 1, 2252633764: 2, 2252768316: 2, 2252768279: 2, 2252782294: 1, 3034708102: 1, 2252633753: 3, 2252782292: 2, 2252782291: 2, 3034708112: 1, 3034708111: 1, 3034708151: 1, 3034708149: 1, 3034708139: 1, 3034708853: 1, 42298786: 1, 267229891: 1, 8096771: 4, 2495729518: 5, 2495729517: 1, 3239156204: 3, 2496163805: 1, 631984925: 1, 119687772: 8, 2742938781: 5, 2742938787: 2, 2730990389: 1, 5150816510: 7, 2730990384: 3, 2730990340: 1, 2730990335: 1, 2736635160: 2, 2730990330: 1, 2730990323: 1, 2736635168: 2, 2

In [ ]:
!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# choose a local (colab) directory to store the data.
local_download_path = 'parking_data'
try:
    os.makedirs(local_download_path)
except: pass

# 2. Auto-iterate using the query syntax
#    https://developers.google.com/drive/v2/web/search-parameters
file_list = drive.ListFile(
    {'q': "'1yR92Tk41yxirhrfiupWSwPaxj5TFzP5i' in parents"}).GetList()  #use your own folder ID here

for f in file_list:
    # 3. Create & download by id.
    print('title: %s, id: %s' % (f['title'], f['id']))
    fname = "parking_data/" + f['title']
    print('downloading to {}'.format(fname))
    f_ = drive.CreateFile({'id': f['id']})
    try:
      f_.GetContentFile(fname)
    except: pass

In [ ]:
# first way to calculate area on the earth surface

!pip install area
from area import area
poly_obj = {'type':'Polygon','coordinates':[[[52.54203249, 13.41329482], [52.54203758, 13.41330569], [52.54206216, 13.4133284], [52.54206483, 13.41332683], [52.54206967, 13.41331855], [52.54207226, 13.41331108], [52.54207127, 13.41330374], [52.54206805, 13.4132987], [52.54205346, 13.413283], [52.54204892, 13.41328021], [52.54204443, 13.41328038], [52.54203642, 13.41328656], [52.54203249, 13.41329482]]]}
area(poly_obj)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for area: filename=area-1.1.1-py3-none-any.whl size=3626 sha256=9c41ea5eda991cc81744205bf56656ac47519244ae7d35f30eabfd620ab18a88
  Stored in directory: /root/.cache/pip/wheels/59/e8/a2/0a9be8239358f5e3b48e1b5cb756f98b0f992b881e1b8b21b2
Successfully built area


13.324528739947908

In [ ]:
# second (and more precise) way for calculating the area on earth surface

def polygon_area(lats, lons, radius = 6378137):
    """
    Computes area of spherical polygon, assuming spherical Earth. 
    Returns result in ratio of the sphere's area if the radius is specified.
    Otherwise, in the units of provided radius.
    lats and lons are in degrees.
    """
    from numpy import arctan2, cos, sin, sqrt, pi, power, append, diff, deg2rad
    lats = np.deg2rad(lats)
    lons = np.deg2rad(lons)

    # Line integral based on Green's Theorem, assumes spherical Earth

    #close polygon
    if lats[0]!=lats[-1]:
        lats = append(lats, lats[0])
        lons = append(lons, lons[0])

    #colatitudes relative to (0,0)
    a = sin(lats/2)**2 + cos(lats)* sin(lons/2)**2
    colat = 2*arctan2( sqrt(a), sqrt(1-a) )

    #azimuths relative to (0,0)
    az = arctan2(cos(lats) * sin(lons), sin(lats)) % (2*pi)

    # Calculate diffs
    # daz = diff(az) % (2*pi)
    daz = diff(az)
    daz = (daz + pi) % (2 * pi) - pi

    deltas=diff(colat)/2
    colat=colat[0:-1]+deltas

    # Perform integral
    integrands = (1-cos(colat)) * daz

    # Integrate 
    area = abs(sum(integrands))/(4*pi)

    area = min(area,1-area)
    if radius is not None: #return in units of radius
        return area * 4*pi*radius**2
    else: #return in ratio of sphere total area
        return area

In [ ]:
cordi = [[52.54203249, 13.41329482], [52.54203758, 13.41330569], [52.54206216, 13.4133284], [52.54206483, 13.41332683], [52.54206967, 13.41331855], [52.54207226, 13.41331108], [52.54207127, 13.41330374], [52.54206805, 13.4132987], [52.54205346, 13.413283], [52.54204892, 13.41328021], [52.54204443, 13.41328038], [52.54203642, 13.41328656], [52.54203249, 13.41329482]]
poly_lon = [x for x,y in cordi]
poly_lat = [y for x,y in cordi]

print(poly_lon)
print(poly_lat)

print(polygon_area(poly_lat, poly_lon))

[52.54203249, 52.54203758, 52.54206216, 52.54206483, 52.54206967, 52.54207226, 52.54207127, 52.54206805, 52.54205346, 52.54204892, 52.54204443, 52.54203642, 52.54203249]
[13.41329482, 13.41330569, 13.4133284, 13.41332683, 13.41331855, 13.41331108, 13.41330374, 13.4132987, 13.413283, 13.41328021, 13.41328038, 13.41328656, 13.41329482]
13.33190691261067
